In [ ]:
!pip install hashformers

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#initialize the hashtag segmentation model
from hashformers import TransformerWordSegmenter as WordSegmenter

ws = WordSegmenter(
    segmenter_model_name_or_path="distilgpt2",
    segmenter_model_type="incremental",
    reranker_model_name_or_path=None,
    segmenter_gpu_batch_size=50000,
)

In [ ]:
import re
from tqdm import tqdm
import time
import torch

file_path = '/kaggle/input/twitter/twitter-datasets/twitter-datasets/train_pos_full.txt'
# Open and read the file
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the contents of the file
    start = time.time()
    for line in file:
        if(count % 1000 == 0): 
            now = time.time()
            diff = now - start
            print(f"{count} lines processed! {diff} time used!")
        torch.cuda.empty_cache()
        #read the line and find all hashtags using re.findall()
        tweet = line.rstrip()
        hashtags = re.findall(r"#\w+", tweet)
        if (len(hashtags) != 0):
            #apply the model to segment
            segmentation = ws.segment(hashtags, use_reranker = False, )
            for i in range(len(segmentation)):
                segmentation[i] = segmentation[i] + "."

            #substitute the original hashtag with segmentation result
            for i in range(len(hashtags)):
                tweet = tweet.replace(hashtags[i], segmentation[i])
        
        tweets.append(tweet)#processed_tweet)
        count += 1
        
#save the processed new tweets
with open('/kaggle/working/updated_train_pos.txt', 'w') as file:
    for s in tweets:
        file.write(s + '\n')

In [ ]:
#same for the test dataset
import re
from tqdm import tqdm
import time
import torch

file_path = '/kaggle/input/twitter/twitter-datasets/twitter-datasets/test_data.txt'
# Open and read the file
tweets = []
count = 0
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the contents of the file
    #start = time.time()
    for line in file:
        if(count % 1000 == 0): 
            #now = time.time()
            #diff = now - start
            #print(f"{count} lines processed! {diff} time used!")
            print(f"{count} lines processed!")
        torch.cuda.empty_cache()
        tweet = line.rstrip()
        hashtags = re.findall(r"#\w+", tweet)
        if (len(hashtags) != 0):
            segmentation = ws.segment(hashtags, use_reranker = False)
            for i in range(len(segmentation)):
                segmentation[i] = segmentation[i] + "."

            for i in range(len(hashtags)):
                tweet = tweet.replace(hashtags[i], segmentation[i])
        
        #processed_tweets = normalizeTweet(tweet)
        tweets.append(tweet)#processed_tweet)
        count += 1
        
print(len(tweets))
with open('/kaggle/working/updated_train_pos.txt', 'w') as file:
    for s in tweets:
        file.write(s + '\n')